In [1]:
import argparse
import os


from dotenv import load_dotenv
load_dotenv()

import wandb
from transformers import RobertaConfig,AutoConfig
from transformers import RobertaForMaskedLM
import torch
import traceback
import logging

from datasets import load_dataset
import util
import time
import util
import torch 

from util import get_checkpoints_hub
from util import DeterministicDataCollatorForLanguageModeling

from multiprocessing import Pool, Queue, Manager

import math
import numpy as np
from transformers import RobertaTokenizerFast,GPT2TokenizerFast, DataCollatorForLanguageModeling,LlamaForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
 

In [ ]:
((939344/100)*32)/1000

In [6]:
939344/1000

939.344

In [2]:
from datasets import load_dataset

ds = load_dataset("anasedova/tulu_3_underspecified_input_errors", split="train")

train.csv:   0%|          | 0.00/60.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/34536 [00:00<?, ? examples/s]

In [3]:
len(ds)

34536

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-2-1124-7B-SFT")
model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-2-1124-7B-SFT")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def preprocess_tulu_confidence(x):
    question = x['question'][0].strip()
    answer = x['response'][0]

    messages = [{"role": "user", "content": f"Answer the question, give ONLY the answer, no other words or explanation:\n\n" + question},
                {"role": "assistant", "content":answer},
                {"role": "user", "content": f"Provide the probability that your answer is correct. Give ONLY the probability between 0.0 and 1.0, no other words or explanation."}]

    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


def preprocess_tulu_errors(data_sample):
    messages = [
        {"role": "user", "content": data_sample["prompt"][0]},
        {"role": "assistant", "content": data_sample["completion"][0]}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)



dataset = load_dataset(
    "anasedova/tulu_3_model_modality_mismatch_errors",
    split="train",
    trust_remote_code=True,
)

ds_tokenized = dataset.with_transform(lambda x : tokenizer([preprocess(x)],return_special_tokens_mask=False, truncation=True, padding="max_length", max_length=4096,return_tensors="pt"))
ds_tokenized = dataset.with_transform(
    lambda x : tokenizer(
        preprocess_tulu_errors(x),
        return_special_tokens_mask=False,
        truncation=True,
        padding="max_length",
        max_length=4096,
        return_tensors="pt"))

In [42]:
def preprocess_tulu_errors(data_sample):
    messages = [
        {"role": "user", "content": data_sample["prompt"][0]},
        {"role": "assistant", "content": data_sample["completion"][0]}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# not sure about the next line
tokenizer.pad_token_id = model.config.eos_token_id

checkpoint = "allenai/OLMo-2-1124-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)
dataset = load_dataset(
    "anasedova/tulu_3_model_modality_mismatch_errors",
    split="train",
    trust_remote_code=True,
)

ds_tokenized = dataset.with_transform(
    lambda x : tokenizer(
        preprocess_tulu_errors(x),
        return_special_tokens_mask=False,
        truncation=True,
        padding="max_length",
        max_length=4096,
        return_tensors="pt"))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
from trl import apply_chat_template

In [ ]:

def preprocess_triviaqa(x):
    question = x['question'][0].strip()
    answer = x['response'][0]

    messages = [{"role": "user", "content": f"Answer the question, give ONLY the answer, no other words or explanation:\n\n" + question},
                {"role": "assistant", "content":answer},
                {"role": "user", "content": f"Provide the probability that your answer is correct. Give ONLY the probability between 0.0 and 1.0, no other words or explanation."}]

    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    


tokenizer.pad_token_id = model.config.eos_token_id    

ds = load_dataset("yuxixia/triviaqa-test-tulu3-query", split="test")
ds_tokenized = ds.with_transform(lambda x : tokenizer([preprocess_triviaqa(x)],return_special_tokens_mask=False, truncation=True, padding="max_length", max_length=4096,return_tensors="pt"))


In [ ]:
ds = load_dataset("yuxixia/triviaqa-test-tulu3-query", split="test")

In [185]:
ds_tokenized

Dataset({
    features: ['question', 'target', 'response', 'confidence', 'question_query', 'response_query', 'confidence_query', 'qa_query', 'pc_query'],
    num_rows: 1000
})

In [ ]:
  0 train[0:100%] pre store_mean

In [186]:
ds[0]

{'question': 'If ultimate means last, what word means second-to-last?',
 'target': 'Penult',
 'response': 'penultimate',
 'confidence': 0.99,
 'question_query': '[]',
 'response_query': '[{\'_index\': \'tulu-3\', \'_id\': \'training_data/tulu-3/train_68.jsonl.gz-3083\', \'_score\': 17.983553, \'_ignored\': [\'text.keyword\'], \'_source\': {\'id\': \'ai2-adapt-dev/evol_codealpaca_heval_decontaminated_107150\', \'source\': \'ai2-adapt-dev/evol_codealpaca_heval_decontaminated\', \'dataset\': None, \'text\': "Develop a Python algorithm that identifies the penultimate highest value within an unordered list of distinct positive integers. Additionally, optimize the algorithm for time complexity and consider edge-case scenarios such as one-element lists. \\narr = [1, 3, 4, 5, 9, 10, 11] Here\'s a Python code solution:\\n\\n```python\\ndef penultimate_highest(numbers):\\n    if len(numbers) < 2:\\n        # If list is empty, there is no penultimate element\\n        # If list only contains one 

In [11]:
ds_tokenized[0]

TypeError: can only concatenate str (not "list") to str

In [46]:
outputs = model.generate(**ds_tokenized[0:10], 
        tokenizer=tokenizer,
        max_new_tokens=20, 
        do_sample=False, 
        # num_beams=1, 
        eos_token_id=tokenizer.eos_token_id,
        # output_scores=True, 
)
print("----")
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
print("----------------------")

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


----
<|user|>
I would like to play a game of chess against you. I will play as white. My first move is pawn to E4. Could you make a visualization of the board and make your move?
<|assistant|>
Sure, though I recommend pasting my visualization into a monospace text editor. It is challenging to get the alignment right. My move is pawn to E5.

```
8 [♜] [♞] [♝] [♛] [♚] [♝] [♞] [♜]
7 [♟︎] [♟︎] [♟︎] [♟︎] [ ] [♟︎] [♟︎] [♟︎]
6 [ ] [ ] [ ] [ ] [ ] [ ] [ ] [ ]
5 [ ] [ ] [ ] [ ] [♟︎] [ ] [ ] [ ]
4 [ ] [ ] [ ] [ ] [♙] [ ] [ ] [ ]
3 [ ] [ ] [ ] [ ] [ ] [ ] [ ] [ ]
2 [♙] [♙] [♙] [♙] [ ] [♙] [♙] [♙]
1 [♖] [♘] [♗] [♕] [♔] [♗] [♘] [♖]
   a   b   c   d   e   f   g   h
```<|assistant|>
] [♟︎] [♟︎] [♟︎] [
----------------------
